In [21]:
import numpy as np
import cv2

In [22]:
cap = cv2.VideoCapture('/content/goldfish.mp4')

In [23]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video frame width is: ", frame_width, "and frame height is: ", frame_height)

Video frame width is:  636 and frame height is:  360


In [24]:
fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
out = cv2.VideoWriter("output.mp4", fourcc, 30, (636, 360))

In [25]:
ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

(360, 636, 3)


In [ ]:
while cap.isOpened():
  diff = cv2.absdiff(frame1, frame2) # difference of change between two frames to detect motion
  gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # grayscaling 
  blur = cv2.GaussianBlur(gray, (5,5), 0) ## bluring to remove noise using gaussian blur filter
  _, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY) ## threshold of detection with a value of 60 on grayscale
  dialated = cv2.dilate(thresh, None, iterations = 10)
  contours, _ = cv2.findContours(dialated, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

  for contour in contours:
    (x_cor, y_cor, W, H) = cv2.boundingRect(contour)
    if cv2.contourArea(contour) < 400:
      continue
    cv2.rectangle(frame1, (x_cor, y_cor), (x_cor+W, y_cor+H), (0, 255, 0), 2)
    cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    img_frame = cv2.resize(frame1, (636, 360))
    out.write(img_frame)
    from google.colab.patches import cv2_imshow
    cv2_imshow(frame1)

    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(60) == 27:
      break

cv2.destroyAllWindows()
cap.release()
out.release()
